<a href="https://colab.research.google.com/github/stazam/ML-hackathon---genomic_benchmarks/blob/main/Introduction_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#pip install genomic_benchmarks --upgrade
%%capture
!pip install genomic_benchmarks

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sys
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, MaxPooling1D, BatchNormalization, Conv1D
from keras import optimizers
from sklearn.model_selection import train_test_split
from pathlib import Path


from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info, list_datasets

#Dataset - human_nontata_promoters 

- testovacia sada obsahuje chybajuce pozorovanie **N**, ktore sa nevyskytuje v treningovom sete, pretoho ho odstranim 

In [2]:
info('human_nontata_promoters', 0)

Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lenghts of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.


,train,test
negative,12355,4119
positive,14742,4915


In [3]:
download_dataset("human_nontata_promoters", version=0)

Unzipping...Done.


PosixPath('/root/.genomic_benchmarks/human_nontata_promoters')

In [4]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / 'human_nontata_promoters'
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=27097,
    class_names=CLASSES)

Found 27097 files belonging to 2 classes.


In [ ]:
#tu mozno pridat PCA alebo nieco na zredukovanie dimenzie. Aj ked 251 je vpohode pre tento dataset

def preprocess_notNN(dat):

  dat_array = np.array(list(dat))

  labels = np.array(dat_array[0][1]).astype('float32')
  dataset = dat_array[0][0]

  sequences_list = []
  for seq in dataset:
    if 'N' not in str(seq):
      sequences_list.append(list(str(seq))[2:-1])

  channels = {'A' : 0,'T' : 1,'C' : 2,'G' : 3}
  
  return np.array(pd.DataFrame(sequences_list).replace(channels)), labels

In [ ]:
X_train, y_train = preprocess_notNN(train_dset)

In [5]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / 'human_nontata_promoters'
CLASSES = [x.stem for x in (SEQ_PATH/'test').iterdir() if x.is_dir()]

test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=9034,
    class_names=CLASSES) 

Found 9034 files belonging to 2 classes.


In [ ]:
X_test, y_test = preprocess_notNN(test_dset)

In [13]:
def preprocess_NN(dat):

  dat_array = np.array(list(dat))

  labels = np.array(dat_array[0][1]).astype('float32')
  dataset = dat_array[0][0]

  sequences_list = []
  for seq in dataset:
    if not 'N' in str(seq):
      sequences_list.append(list(str(seq))[2:-1])

  samples_size = len(sequences_list)
  sequence_size = min([len(x) for x in sequences_list])
  ohe = np.zeros((samples_size, sequence_size, 4))
  channels = {'A' : 0,'T' : 1,'C' : 2,'G' : 3}

  for index, sequence in enumerate(sequences_list):
    for pos, nucleotide in enumerate(sequence):
        ohe[index, pos, channels[nucleotide]] = 1
  
  return ohe, labels, sequence_size

In [15]:

X_train, y_train, sequence_size = preprocess_NN(train_dset)
X_test, y_test, _ = preprocess_NN(test_dset)

skusit ensemble s obycajnymi modeli a jednou CNN alebo potom este pridat jednu RNN a vyskusat to s nou

1. samostatne modey bez NN
2. modely s 1-nou NN - najprv CNN, potom RNN
3. modely s 2-ma NN - CNN, RNN
4. pouzit PCA na zredukovanie dimenzie

In [8]:
from google.colab import drive
#drive.mount('/content/drive',  force_remount=True)
drive.mount('/content/drive',  force_remount=True)


Mounted at /content/drive


In [10]:
#sys.path.append('/content/drive/MyDrive/CEITEC/Python_files')
#sys.path.append('/content/drive/MyDrive/Hockey_prediction/Python_script')
sys.path.append('/content/drive/MyDrive/ML_Hackathon/')

from help_functions import *

In [22]:
def CNN_win_model(sequence_size):
  model = Sequential([
        Conv1D(filters = 32, kernel_size=8, padding='same', activation = 'relu', input_shape=(sequence_size, 4)),
        Dropout(0.5),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

  model.summary()

  return model

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_2 (Conv1D)           (None, 251, 32)           1056      
                                                                 
 dropout_4 (Dropout)         (None, 251, 32)           0         
                                                                 
 flatten_2 (Flatten)         (None, 8032)              0         
                                                                 
 dense_9 (Dense)             (None, 128)               1028224   
                                                                 
 dense_10 (Dense)            (None, 128)               16512     
                                                                 
 dense_11 (Dense)            (None, 64)                8256      
                                                                 
 dense_12 (Dense)            (None, 64)               

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, train_size = 0.8, random_state = 20)
model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

models_data = model.fit(X_train, y_train, batch_size=32, epochs=8, validation_data = (X_test, y_test))

Epoch 1/8
434/434 [==============================] - 11s 23ms/step - loss: 0.4765 - accuracy: 0.7741 - val_loss: 0.3560 - val_accuracy: 0.8475
Epoch 2/8
434/434 [==============================] - 10s 22ms/step - loss: 0.3793 - accuracy: 0.8332 - val_loss: 0.3294 - val_accuracy: 0.8570
Epoch 3/8
434/434 [==============================] - 10s 22ms/step - loss: 0.3472 - accuracy: 0.8494 - val_loss: 0.3744 - val_accuracy: 0.8342
Epoch 4/8
434/434 [==============================] - 10s 22ms/step - loss: 0.3214 - accuracy: 0.8603 - val_loss: 0.3293 - val_accuracy: 0.8671
Epoch 5/8
434/434 [==============================] - 10s 22ms/step - loss: 0.3075 - accuracy: 0.8710 - val_loss: 0.3262 - val_accuracy: 0.8645
Epoch 6/8
434/434 [==============================] - 10s 22ms/step - loss: 0.2929 - accuracy: 0.8827 - val_loss: 0.3317 - val_accuracy: 0.8688
Epoch 7/8
434/434 [==============================] - 10s 22ms/step - loss: 0.2855 - accuracy: 0.8841 - val_loss: 0.3469 - val_accuracy: 0.8642

In [24]:
metrics = model.evaluate(X_test, y_test, verbose=0)
print('model evaluation on unknown dataset [loss, accuracy]:', metrics)

model evaluation on unknown dataset [loss, accuracy]: [0.35315585136413574, 0.8702796101570129]


In [ ]:
model.add(BatchNormalization())
  model.add(MaxPooling1D())